In [137]:
import pandas as pd
import os
import numpy as np
import glob
import matplotlib.pyplot as plt


In [138]:
root_path = 'D:\\Career\\Hackathons\\Ocean_Dimitra_Bounty'
crop_statistics_path = os.path.join(root_path, 'Input', 'DISTRICT WISE CROP STATISTICS MADHYA PRADESH')


In [139]:
crop_statistics_filename = os.path.join(crop_statistics_path,'DISTRICT_WISE_CROP_STATISTICS.xlsx')
crop_statistics_df = pd.read_excel(crop_statistics_filename, sheet_name=0, skiprows=[0,1,3])

In [140]:
rename_dict = {'State/Crop/District':'district'
               , 'Area (Hectare)':'area_hectare'
               , 'Production (Tonnes)':'production_tonnes'
               , 'Yield (Tonnes/Hectare)':'yield_tph'}
crop_statistics_df.rename(columns=rename_dict, inplace=True)
crop_statistics_df.columns = crop_statistics_df.columns.str.lower()

In [141]:
crop_statistics_df['crop'] = np.NaN
crop_statistics_df_copy = crop_statistics_df.copy()

In [142]:
for idx1, row1 in crop_statistics_df_copy.iterrows():
    if 'Total' in str(row1['district']):
        crop_statistics_df = crop_statistics_df.drop(idx1)
        continue
    if 'Total' in str(row1['season']):
        crop_statistics_df = crop_statistics_df.drop(idx1)
        continue
    
    if row1.isna().sum() > 4:
        crop_statistics_df.loc[crop_statistics_df.index>idx1, 'crop'] = row1['district']
        crop_statistics_df = crop_statistics_df.drop(idx1)
        
crop_statistics_df = crop_statistics_df.fillna(method='ffill')

In [143]:
crop_statistics_df.loc[crop_statistics_df['yield_tph']=='Bales/Hectare', 'yield_tph'] = 0
crop_statistics_df.loc[crop_statistics_df['production_tonnes']=='Bales*', 'production_tonnes'] = 0

crop_statistics_df = crop_statistics_df.astype({'area_hectare':float, 'production_tonnes':float, 'yield_tph':float})

crop_statistics_df['district'] = crop_statistics_df['district'].str.split('.').str[1]


In [144]:
crop_statistics_df.describe()

,area_hectare,production_tonnes,yield_tph
count,29909.000000,2.990900e+04,29909.000000
mean,16453.945316,2.741738e+04,3.660036
std,43729.565909,9.495561e+04,8.075567
min,1.000000,0.000000e+00,0.000000
25%,97.000000,1.150000e+02,0.455556
50%,976.000000,1.300000e+03,0.940000
75%,8464.000000,1.044100e+04,2.000000
max,537610.000000,4.018000e+06,138.421053


#### Yearly growth in cropping area for each district

In [145]:
crop_area_yearly = crop_statistics_df.groupby(by=['district','year'], as_index=False).agg({'area_hectare':'mean'})
# crop_area_yearly.plot(x='district', y='area_hectare')
# plt.show()

In [146]:
for district in crop_area_yearly.district.unique():
    crop_area_yearly.loc[crop_area_yearly['district']==district, 'area_yoy_growth'] = crop_area_yearly.loc[crop_area_yearly['district']==district, 'area_hectare'].pct_change(periods=1)

crop_area_yearly = crop_area_yearly.replace([np.inf, -np.inf, np.NaN], 0)

area_yoy_growth_df = crop_area_yearly.groupby('district', as_index=False).agg({'area_yoy_growth':'mean'})

topdistrict_area_yoy_growth = area_yoy_growth_df.sort_values('area_yoy_growth', ascending=False).iloc[:5]['district'].tolist()
topdistrict_area_yoy_growth

['ANUPPUR', 'RAIPUR', 'DURG', 'BILASPUR', 'SURGUJA']

#### Yearly growth in production for each district

In [147]:
production_yearly = crop_statistics_df.groupby(by=['district','year'], as_index=False).agg({'production_tonnes':'mean'})

In [148]:
for district in production_yearly.district.unique():
    production_yearly.loc[production_yearly['district']==district, 'production_yoy_growth'] = production_yearly.loc[production_yearly['district']==district, 'production_tonnes'].pct_change(periods=1)

production_yearly = production_yearly.replace([np.inf, -np.inf, np.NaN], 0)

production_yoy_growth_df = production_yearly.groupby('district', as_index=False).agg({'production_yoy_growth':'mean'})

topdistrict_production_yoy_growth = production_yoy_growth_df.sort_values('production_yoy_growth', ascending=False).iloc[:5]['district'].tolist()
topdistrict_production_yoy_growth

['DURG', 'RAIPUR', 'BILASPUR', 'DHAMTARI', 'BASTAR']

#### Yearly growth in yield for each district

In [149]:
yield_yearly = crop_statistics_df.groupby(by=['district','year'], as_index=False).agg({'yield_tph':'mean'})

In [150]:
for district in yield_yearly.district.unique():
    yield_yearly.loc[yield_yearly['district']==district, 'yield_yoy_growth'] = yield_yearly.loc[yield_yearly['district']==district, 'yield_tph'].pct_change(periods=1)

yield_yearly = yield_yearly.replace([np.inf, -np.inf, np.NaN], 0)

yield_yoy_growth_df = yield_yearly.groupby('district', as_index=False).agg({'yield_yoy_growth':'mean'})

topdistrict_yield_yoy_growth = yield_yoy_growth_df.sort_values('yield_yoy_growth', ascending=False).iloc[:5]['district'].tolist()
topdistrict_yield_yoy_growth

['KOREA', 'BILASPUR', 'DURG', 'MAHASAMUND', 'SURGUJA']

#### Crop Statistics

In [151]:
crop_statistics_df['crop'].unique()

array(['Arhar/Tur', 'Bajra', 'Banana', 'Barley', 'Cashewnut',
       'Castor seed', 'Coriander', 'Cowpea(Lobia)', 'Dry chillies',
       'Garlic', 'Ginger', 'Gram', 'Groundnut', 'Horse-gram', 'Jowar',
       'Khesari', 'Linseed', 'Maize', 'Masoor', 'Moong(Green Gram)',
       'Niger seed', 'Onion', 'Other  Rabi pulses', 'Other Cereals',
       'Other Kharif pulses', 'Peas & beans (Pulses)', 'Potato', 'Ragi',
       'Rapeseed &Mustard', 'Rice', 'Safflower', 'Sannhamp', 'Sesamum',
       'Small millets', 'Soyabean', 'Sugarcane', 'Sunflower',
       'Sweet potato', 'Tobacco', 'Turmeric', 'Urad', 'Wheat'],
      dtype=object)